# Postdam PM2.5 Exponential Smooting Forcasting 

* Between 2013 and 2023, data collected by DEBB021 was used.
* To increase the accuracy of PM2.5 data estimation, NO2, O3, SO2, PM10 pollutant gas data accepted by the EEA was added.


In [1]:
# imports
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np, pandas as pd

In [2]:
# import src
import model_base as mb
import exponential_smoothing as exps

## Data Exploration

* Load Data


In [3]:
df = mb.get_cleaned_datetime_df()
df.head()

,Start_Timestamp,End_Timestamp,Start,End,PM2.5-Pollutant,PM2.5-Value,PM2.5-Unit,PM2.5-Validity,PM2.5-Verification,PM10-Pollutant,...,O3-Pollutant,O3-Value,O3-Unit,O3-Validity,O3-Verification,SO2-Pollutant,SO2-Value,SO2-Unit,SO2-Validity,SO2-Verification
0,1356998400,1357002000,2013-01-01 00:00:00,2013-01-01 01:00:00,6001,71.04,ug.m-3,1,1,5,...,7,43.17,ug.m-3,1,1,1,12.18,ug.m-3,1,1
1,1357002000,1357005600,2013-01-01 01:00:00,2013-01-01 02:00:00,6001,20.52,ug.m-3,1,1,5,...,7,57.15,ug.m-3,1,1,1,4.65,ug.m-3,1,1
2,1357005600,1357009200,2013-01-01 02:00:00,2013-01-01 03:00:00,6001,9.56,ug.m-3,1,1,5,...,7,63.31,ug.m-3,1,1,1,1.33,ug.m-3,1,1
3,1357009200,1357012800,2013-01-01 03:00:00,2013-01-01 04:00:00,6001,9.45,ug.m-3,1,1,5,...,7,63.18,ug.m-3,1,1,1,1.33,ug.m-3,1,1
4,1357012800,1357016400,2013-01-01 04:00:00,2013-01-01 05:00:00,6001,13.02,ug.m-3,1,1,5,...,7,61.70,ug.m-3,1,1,1,1.33,ug.m-3,1,1


In [4]:
mb.set_start_index(df, 'Start')
df.index = pd.to_datetime(df.index)
df.head()

,Start_Timestamp,End_Timestamp,End,PM2.5-Pollutant,PM2.5-Value,PM2.5-Unit,PM2.5-Validity,PM2.5-Verification,PM10-Pollutant,PM10-Value,...,O3-Pollutant,O3-Value,O3-Unit,O3-Validity,O3-Verification,SO2-Pollutant,SO2-Value,SO2-Unit,SO2-Validity,SO2-Verification
Start,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1356998400,1357002000,2013-01-01 01:00:00,6001,71.04,ug.m-3,1,1,5,88.96,...,7,43.17,ug.m-3,1,1,1,12.18,ug.m-3,1,1
2013-01-01 01:00:00,1357002000,1357005600,2013-01-01 02:00:00,6001,20.52,ug.m-3,1,1,5,25.17,...,7,57.15,ug.m-3,1,1,1,4.65,ug.m-3,1,1
2013-01-01 02:00:00,1357005600,1357009200,2013-01-01 03:00:00,6001,9.56,ug.m-3,1,1,5,11.97,...,7,63.31,ug.m-3,1,1,1,1.33,ug.m-3,1,1
2013-01-01 03:00:00,1357009200,1357012800,2013-01-01 04:00:00,6001,9.45,ug.m-3,1,1,5,11.73,...,7,63.18,ug.m-3,1,1,1,1.33,ug.m-3,1,1
2013-01-01 04:00:00,1357012800,1357016400,2013-01-01 05:00:00,6001,13.02,ug.m-3,1,1,5,15.88,...,7,61.70,ug.m-3,1,1,1,1.33,ug.m-3,1,1


# Exponential Smooting

Exponential smoothing is a family of forecasting methods that apply weighted averages of past observations to forecast future values. The weights decrease exponentially as the observations get older, hence the name. Exponential smoothing methods can be adapted to both seasonality and trend, but they do not require either to be present in the time series data.


**Simple Exponential Smoothing:** This method is suitable for time series without trend and seasonality. It forecasts the future values based on a weighted average of past observations, with the weights declining exponentially as the observations get older.

**Holt’s Linear Trend Method (Double Exponential Smoothing):** This method extends simple exponential smoothing to capture linear trends in the data. It uses two smoothing equations: one for the level (the average value) and one for the trend.

**Holt-Winters’ Seasonal Method (Triple Exponential Smoothing):** This method further extends exponential smoothing to capture seasonality in addition to level and trend. It incorporates a third smoothing equation for the seasonal component. There are two variations of Holt-Winters' method: the additive version for time series with a stable seasonal pattern regardless of the level, and the multiplicative version for when the seasonal pattern varies with the level of the time series.

* For a time series with no trend and no seasonality, simple exponential smoothing is appropriate.
* For a time series with a trend but no seasonality, Holt’s method is appropriate.
* For a time series with both trend and seasonality, Holt-Winters’ method is appropriate.


In [5]:
# Defining Target and feature variables
X,y = mb.define_target_features(df)

## Principle Component Analysis (PCA)
Principal Component Analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components.

In [6]:
# Apply PCA on the scaled data
# pca = mb.init_pca()
# principalComponents = pca.fit_transform(X)
# principalDf = pd.DataFrame(data=principalComponents, index=df.index)

# print(principalDf)

In [7]:
# Combine PCA components with the target
# finalDf = pd.concat([principalDf, y], axis=1)

## Splitting Data 

Train, Validation and Test data

In [8]:
train_data, validation_data, test_data = mb.split_data(df)

In [9]:
# Get the features
X_train, X_val, X_test = mb.extract_features(train_data, validation_data, test_data)

In [10]:
# Extract the target variable
y_train, y_val, y_test = mb.extract_target(train_data, validation_data, test_data)

## Model Creation
* Initialize Linear Regression Model
* Train model

In [11]:
# Initialize and train the exponential smoothing model
model = exps.init_fit_model(train_data)

# Evaluation 

## With Validation Data

Error metrics MAE, MSE, RMSE, MASE, MAPE

* Regarding the MASE metric, calculating it requires a baseline prediction model for the time series, which is typically done by using the last observed value to predict the next (in the simplest case) or using more complex methods like ARIMA for one-step ahead forecasting. This is not included in the above script as it would require additional steps to implement the naive forecasting method for a time series.

### Predict Validation


In [ ]:
# Make predictions on the validation set
# y_val_pred = model.predict(start=validation_data.index[0], end=validation_data.index[-1])
# print(y_val_pred)

y_val_pred = model.forecast(len(validation_data))
print(y_val_pred)

print(validation_data.index[0])
print(validation_data.index[-1])

In [ ]:
# Error Metric

mb.evolve_error_metrics(y_val,y_val_pred)
mb.naive_mean_absolute_scaled_error(y_val,y_val_pred)

## With Test Data


In [ ]:
# Predict on the test set
# y_test_pred = model.predict(start=test_data.index[0], end=test_data.index[-1])

y_test_pred = model.forecast(len(test_data))
print(y_test_pred)


print(test_data.index[0])
print(test_data.index[-1])
print(y_test_pred)



# Error Metric
mb.evolve_error_metrics(y_test ,y_test_pred)
mb.naive_mean_absolute_scaled_error(y_test,y_test_pred)

## Plot Table 


In [ ]:
mb.plot_pm_true_predict(validation_data, y_val_pred, 'Validation')
mb.plot_pm_true_predict(test_data, y_test_pred, 'Test')

# HyperPramater Tuning

Linear Regression typically has fewer hyperparameters than other models like neural networks or ensemble models. However, there are still some aspects of the model that you can adjust. For instance, you can apply regularization, which can be considered a form of hyperparameter tuning. The most common types of regularized linear regression are Ridge Regression (L2 regularization) and Lasso Regression (L1 regularization).

In [17]:
from itertools import product
from statsmodels.tsa.api import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Define your hyperparameter grid
trend_options = ['add', 'mul', None]
seasonal_options = ['add', 'mul', None]
seasonal_periods_options = [12]  # example for monthly data with yearly seasonality
damped_options = [True, False]

# Cartesian product of the hyperparameter grid
param_grid = list(product(trend_options, seasonal_options, seasonal_periods_options, damped_options))

# Keep track of the best configuration and corresponding MSE
best_mse = float("inf")
best_config = None
best_model = None

# Perform grid search
for params in param_grid:
    trend, seasonal, seasonal_periods, damped = params

    # Skip if both trend and seasonal are None
    if trend is None and seasonal is None:
        continue
    
    try:
        # Fit the model with the current set of hyperparameters
        model = ExponentialSmoothing(
            train_data['PM2.5-Value'], 
            seasonal_periods=seasonal_periods, 
            trend=trend, 
            seasonal=seasonal,
            damped=damped
        ).fit(use_boxcox=True)

        # Forecast on the validation set
        val_predictions = model.forecast(len(validation_data))

        # Calculate the MSE for this model configuration
        mse = mean_squared_error(validation_data['PM2.5-Value'], val_predictions)

        # Check if this configuration gives us a lower MSE than what we've seen so far
        if mse < best_mse:
            best_mse = mse
            best_config = params
            best_model = model

    except Exception as e:
        print(f"Error with configuration {params}: {e}")

# Output the best performing model configuration
print(f"Best configuration: Trend: {best_config[0]}, Seasonal: {best_config[1]}, Seasonal Periods: {best_config[2]}, Damped: {best_config[3]}")
print(f"Best MSE: {best_mse}")

/Users/tugcekonuklar/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
/Users/tugcekonuklar/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
/Users/tugcekonuklar/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
/Users/tugcekonuklar/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
/Users/tugcekonuklar/opt/anaconda3/lib/p

Error with configuration (None, 'add', 12, True): Can only dampen the trend component


/Users/tugcekonuklar/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
/Users/tugcekonuklar/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'


Error with configuration (None, 'mul', 12, True): Can only dampen the trend component
Best configuration: Trend: mul, Seasonal: add, Seasonal Periods: 12, Damped: True
Best MSE: 114.97180825533549
